In [1]:
import os
import numpy as np
from PIL import Image, ImageTk
from io import BytesIO

import requests
from tkinter import Tk, Label, Canvas, Entry, Button, NW

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image as image_utils


In [2]:
model = Sequential()

#------- input layer----

model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
# -----------------

#--------- hidden layer-------
# here 64 is for ascending order in hidden layer (..., 32, 64, 128, 512, ...)
# for descending order in hidden layer (..., 512, 128, 64, 32, ...)

model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
#-------------------------


#--- create bridge-------
# (for connecting b2n hidden layer and output layer)

model.add(Flatten())
#------------------

#----- output layer-----------
# for avoiding problems we will not create output layer directly, So, create FFNN
# 128 should be the same or equal to avoid collision

model.add(Dense(128, activation = "relu")) 
model.add(Dense(3, activation = "softmax")) 

In [3]:
model.compile(optimizer= "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

In [4]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  shear_range = 0.2, 
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

In [5]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [6]:
training_datasets = train_datagen.flow_from_directory("dataset/training_set", target_size = (128, 128), batch_size = 32, class_mode = "categorical") # local directory
testing_datasets = test_datagen.flow_from_directory("dataset/test_set", target_size = (128, 128), batch_size = 32, class_mode = "categorical") 

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [7]:
model.fit(training_datasets, steps_per_epoch= 800/32, epochs = 10, validation_data = testing_datasets, validation_steps = 200/32)

Epoch 1/10
25/25 [==============================] - 42s 2s/step - loss: 1.1046 - accuracy: 0.3812 - val_loss: 1.0492 - val_accuracy: 0.4062
Epoch 2/10
25/25 [==============================] - 32s 1s/step - loss: 1.0191 - accuracy: 0.4600 - val_loss: 0.8957 - val_accuracy: 0.5446
Epoch 3/10
25/25 [==============================] - 26s 1s/step - loss: 0.9319 - accuracy: 0.5350 - val_loss: 0.8406 - val_accuracy: 0.5223
Epoch 4/10
25/25 [==============================] - 25s 985ms/step - loss: 0.9524 - accuracy: 0.5000 - val_loss: 0.9088 - val_accuracy: 0.5714
Epoch 5/10
25/25 [==============================] - 23s 932ms/step - loss: 0.8765 - accuracy: 0.5850 - val_loss: 0.7700 - val_accuracy: 0.6116
Epoch 6/10
25/25 [==============================] - 21s 855ms/step - loss: 0.8713 - accuracy: 0.5850 - val_loss: 0.7500 - val_accuracy: 0.6607
Epoch 7/10
25/25 [==============================] - 21s 858ms/step - loss: 0.8089 - accuracy: 0.6137 - val_loss: 0.7166 - val_accuracy: 0.6875
Epoch 8/

In [12]:
# save model
model.save("model/model.h5")
model.save_weights("model/model_weights.h5")

In [13]:
image_width, image_height = 128, 128
model_path = "model/model.h5"
model_weight_path = "model/model_weights.h5"

final_model = load_model(model_path)
final_model.load_weights(model_weight_path)

In [14]:
url = ''
window = Tk()
window.title("Image Classification Using CNN")
window.geometry("800x800")

label = Label(window, text = "Please enter Url", font = ("Halvetica", 16))
label.pack()


def Enter_Url():
    global url
    label.configure()
    url = (User_input.get())
    print(url)
    
    
    response = requests.get(url)
    test_image = Image.open(BytesIO(response.content))
    put_image = test_image.resize((400, 400))
    test_image = test_image.resize((128, 128))
    
    
    img = ImageTk.PhotoImage(put_image)
    pic = Label(image = img)
    pic.pack()
    
    pic.image = img
    test_image = image_utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    
    results = model.predict_on_batch(test_image)
    
    if results[0][0] == 1:
        res = "French Fries"
    elif results[0][1] == 1:
        res = "Pizza"
    elif results[0][2] == 1:
        res = "Samosa"
        
        
    output = Label(window, text = "Predicted Results: "+ res, font = ("Halvetica", 16))
    output.pack()

In [ ]:
User_input = Entry()
User_input.pack()

button = Button(window, text = "Detect", font = ("Halvetica", 16), command = Enter_Url)
button.pack()
window.mainloop()